In [ ]:
import os
path = '/content/drive/MyDrive/VLSP_ReINTEL'
#you should change this path to your project folder path
os.chdir(path)

In [ ]:
!pip install pyvi

import numpy as np
from tqdm import tqdm

np.random.seed(42)
import pandas as pd
from pyvi import ViTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import BatchNormalization, MaxPooling1D, Conv1D#, Merge
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, LSTM, Bidirectional
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras import backend as K
from keras.models import model_from_json
from keras.models import load_model

     |████████████████████████████████| 8.5MB 7.2MB/s 
     |████████████████████████████████| 747kB 41.5MB/s 


In [ ]:
#download fasttext 
#! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz

--2020-12-07 06:31:17--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1235219084 (1.1G) [binary/octet-stream]
Saving to: ‘cc.vi.300.vec.gz’

cc.vi.300.vec.gz    100%[===================>]   1.15G  40.8MB/s    in 25s     

2020-12-07 06:31:43 (46.4 MB/s) - ‘cc.vi.300.vec.gz’ saved [1235219084/1235219084]



In [ ]:
#extract fasttext
'''
import gzip
import shutil
with gzip.open('cc.vi.300.vec.gz', 'rb') as f_in:
    with open('cc.vi.300.vec', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        '''

In [ ]:
#download phow2v
#! wget https://public.vinai.io/word2vec_vi_words_300dims.zip

--2020-12-07 06:33:47--  https://public.vinai.io/word2vec_vi_words_300dims.zip
Resolving public.vinai.io (public.vinai.io)... 99.86.35.23, 99.86.35.91, 99.86.35.72, ...
Connecting to public.vinai.io (public.vinai.io)|99.86.35.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2315583928 (2.2G) [application/zip]
Saving to: ‘word2vec_vi_words_300dims.zip’

word2vec_vi_words_3 100%[===================>]   2.16G  14.7MB/s    in 2m 33s  

2020-12-07 06:36:23 (14.4 MB/s) - ‘word2vec_vi_words_300dims.zip’ saved [2315583928/2315583928]



In [ ]:
'''
from zipfile import ZipFile
zip = ZipFile('/content/word2vec_vi_words_300dims.zip')
zip.extractall()
'''

In [ ]:
#download phow2v syllables
#! wget https://public.vinai.io/word2vec_vi_syllables_300dims.zip

--2020-12-07 06:39:53--  https://public.vinai.io/word2vec_vi_syllables_300dims.zip
Resolving public.vinai.io (public.vinai.io)... 99.86.35.91, 99.86.35.98, 99.86.35.23, ...
Connecting to public.vinai.io (public.vinai.io)|99.86.35.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1422871284 (1.3G) [application/zip]
Saving to: ‘word2vec_vi_syllables_300dims.zip’

word2vec_vi_syllabl 100%[===================>]   1.32G  14.6MB/s    in 96s     

2020-12-07 06:41:30 (14.1 MB/s) - ‘word2vec_vi_syllables_300dims.zip’ saved [1422871284/1422871284]



In [ ]:
'''
zip = ZipFile('word2vec_vi_syllables_300dims.zip')
zip.extractall()
'''

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/BERT/SA/VSLP_data/public_train.csv')
train['post_message']=train['post_message'].fillna('none')
public_test = pd.read_csv('/content/drive/MyDrive/BERT/SA/VSLP_data/public_test.csv')
private_test = pd.read_csv('/content/drive/MyDrive/BERT/SA/VSLP_data/final_private_test_dropped_no_label - final_private_test_dropped_no_label.csv')

In [ ]:
X_train = train["post_message"].fillna("none").values
y_train = train[['label']].values
X_test = private_test["post_message"].fillna("none").values

In [ ]:
def preprocess(text):
    text = text.split(" ")
    return text

max_features =  30000
maxlen = 500
embed_size = 300
batch_size = 32
epochs = 5
l2_reg = .00001
filter_sizes = [3,4,5]
num_filters = 32
EMBEDDING_FILE = 'cc.vi.300.vec'
#EMBEDDING_FILE = 'word2vec_vi_words_300dims.txt' 
#you can change path here to load another word embedding pretrained model

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test)) 
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)


embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embed_size))

max_features = num_words

for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

import tensorflow as tf

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=233)

checkpoint = ModelCheckpoint('w2v_balanced_v1.hdf5',
                             monitor='val_acc',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

def get_model():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Reshape((maxlen, embed_size, 1))(x)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                    activation='elu')(x)

    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)

    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    z = Flatten()(z)
    z = Dropout(0.35)(z)
    outp = Dense(1, activation="sigmoid")(z)

  

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',f1_m,precision_m, recall_m])

    return model



In [ ]:
model = get_model()
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 500, 300)     5555100     input_6[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 500, 300)     0           embedding_5[0][0]                
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 500, 300, 1)  0           spatial_dropout1d_3[0][0]        
_______________________________________________________________________________________

In [ ]:
history = model.fit(X_tra, y_tra, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/3
122/122 [==============================] - 92s 755ms/step - loss: 0.4320 - acc: 0.8347 - f1_m: 0.0617 - precision_m: 0.1510 - recall_m: 0.0423 - val_loss: 0.3698 - val_acc: 0.8388 - val_f1_m: 0.0988 - val_precision_m: 0.2903 - val_recall_m: 0.0608
Epoch 2/3
122/122 [==============================] - 92s 752ms/step - loss: 0.3228 - acc: 0.8647 - f1_m: 0.3934 - precision_m: 0.6717 - recall_m: 0.3095 - val_loss: 0.3289 - val_acc: 0.8552 - val_f1_m: 0.2983 - val_precision_m: 0.5070 - val_recall_m: 0.2345
Epoch 3/3
122/122 [==============================] - 92s 752ms/step - loss: 0.2494 - acc: 0.8970 - f1_m: 0.5844 - precision_m: 0.8108 - recall_m: 0.4917 - val_loss: 0.3047 - val_acc: 0.8706 - val_f1_m: 0.4903 - val_precision_m: 0.6862 - val_recall_m: 0.4136


In [ ]:
from keras.models import load_model
y_pred = model.predict(x_test, batch_size=32)

model_json = model.to_json()
with open("cnn_.json", "w") as json_file:
    json_file.write(model_json)

model.save('cnn_.h5')
#new_model = load_model('/content/drive/MyDrive/VSLP_ReINTEL/text cnn/CNN.h5')


In [ ]:
y_pred = model.predict(x_test, batch_size=32)

In [ ]:
results = pd.DataFrame({'id':private_test['id']})
results['probability']=y_pred
results.to_csv('results.csv',index=False,header=False)